In [ ]:
import pandas as pd

# **Coletando Dados de Fontes Oficiais**

Utilizamos fontes da Johns Hopkins CSSEGIS 

In [ ]:
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
recovered.describe()

In [ ]:
confirmed.describe()

In [ ]:
confirmed.last_valid_index

In [ ]:
confirmed.shape

In [ ]:
len(confirmed["Country/Region"].unique())

In [ ]:
confirmed.iloc[0]

In [ ]:
confirmed.set_index("Province/State").loc["Beijing"]

In [ ]:
confirmed["Country/Region"].value_counts()

In [ ]:
confirmed_by_country = confirmed.groupby("Country/Region").sum()
confirmed_by_country.head()

In [ ]:
confirmed_by_country.loc["Brazil"]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
confirmed_by_country.loc["Brazil"][2:].plot()
plt.title("Casos covid-19 confirmados no Brasil")
plt.show()

In [ ]:
new_cases_brazil = confirmed_by_country.loc["Brazil"][2:].diff().dropna()


In [ ]:
plt.figure(figsize=(15,5))
new_cases_brazil.plot()
plt.title("Casos novos de covid-19, confirmados no Brasil")
plt.show()

In [ ]:
last_date = confirmed_by_country.iloc[:,-1].name 
last_date


In [ ]:
fig = plt.figure(figsize=(15,5))
ax = confirmed_by_country[last_date].sort_values(ascending=False)[:20].plot(kind='bar')
ax = plt.title("Covid-19, total new cases until yesterday, " + last_date, loc='left', pad='30',fontsize = 25)
ax = plt.show()

In [ ]:
display(deaths.head())
display(recovered.head())


In [ ]:
def latest_by_country(data):
  return data.groupby("Country/Region").sum().iloc[:,-1]

latest_by_country(deaths)

In [ ]:
informations = [latest_by_country(confirmed), 
               latest_by_country(deaths),
               latest_by_country(recovered)]
combined = pd.concat(informations, axis = 1)
combined.columns = ["confirmed", "deaths", "recovered"]
combined.head()

# Calculando a taxa de letalidade da doença

Considera o número total de mortos em relação ao número de casos confirmados.

In [ ]:
sum_up = combined.sum()
letality_rate_1 = sum_up["deaths"]/sum_up["confirmed"]
print(sum_up["deaths"])
print(sum_up["confirmed"])
print(letality_rate_1*100)

# Calculando a taxa de letalidade por país

In [ ]:
letality = combined["deaths"]/combined["confirmed"]*100
letality

In [ ]:
letality = combined["deaths"]/combined["confirmed"]*100
combined["letality_rate_1"] = letality
combined.head(10)

#Calculando uma segunda taxa de letalidade, considerando como total o número de mortos somados aos recuperados.

Como a base de dados de **Recuperados** deixou de ser atualizada diariamente a partir de 5/08/2021, iremos fazer o calculo com os dados normatizados até esta data. 


In [ ]:
recovered.loc[:,:'8/5/21'].head(50)

###Criando as bases de valores acumulados até '8/4/21'

In [ ]:
confirmed = confirmed.loc[:, :'8/4/21']
deaths = deaths.loc[:, :'8/4/21']
recovered = recovered.loc[:, :'8/4/21']

In [ ]:
informations = [latest_by_country(confirmed), 
               latest_by_country(deaths),
               latest_by_country(recovered)]
combined = pd.concat(informations, axis = 1)
combined.columns = ["confirmed", "deaths", "recovered"]
combined.head()

In [ ]:
letality_rate_1 = combined["deaths"]/combined["confirmed"]*100
letality_rate_2 = combined["deaths"]/(combined["recovered"] + combined["deaths"])*100

combined["letality_rate_1"] = letality_rate_1
combined["letality_rate_2"] = letality_rate_2


In [ ]:
combined.index[0:49], combined.index[50:99],combined.index[100:149],combined.index[150:198]   

In [ ]:
combined.sort_values("letality_rate_2", ascending=False).head(10)

In [ ]:
fig1 = plt.figure(figsize=(15,5))
ax1 = letality_rate_1.sort_values(ascending=False)[:15].plot(kind='bar', color ='green')
ax1 = plt.title("Covid-19, letality rate 1 until 8/4/21", loc='left', pad='30',fontsize = 25)
plt.show()

fig2 = plt.figure(figsize=(15,5))
ax2 = letality_rate_2.sort_values(ascending=False)[:15].plot(kind='bar')
ax2 = plt.title("Covid-19, letality rate 2 until 8/4/21", loc='left', pad='30',fontsize = 25)
plt.show()

In [ ]:
combined [["confirmed", "letality_rate_1"]].plot()

Como a visualização, usando a função plot nao ficou clara, vamos utilizar os gráficos do **Seaborn**

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(data = combined, x = "confirmed", y = "letality_rate_1")

In [ ]:
sns.scatterplot(data = combined.query("confirmed >50000"), x = "confirmed", y = "letality_rate_1", color ='orange' )

In [ ]:
sns.displot(combined.query("confirmed > 50000")["letality_rate_1"])
plt.show()

In [ ]:
def latest_by_country_at(data, date):
  return data.groupby("Country/Region").sum()[date]

In [ ]:
informations = [latest_by_country_at(confirmed, '2/20/20'), latest_by_country_at(deaths, '2/20/20'), latest_by_country_at(recovered, '2/20/20')]
combined_2_20_20 = pd.concat(informations, axis=1)
combined_2_20_20.columns = ["confirmed", "deaths", "recovered"]

sum_up = combined_2_20_20.loc["China"]

letality_rate_1 = sum_up["deaths"]/sum_up["confirmed"] * 100
letality_rate_2 = sum_up["deaths"]/(sum_up["recovered"]+sum_up["deaths"]) * 100

print(letality_rate_1)
print(letality_rate_2)


In [ ]:
informations = [latest_by_country_at(confirmed, '2/8/20'), latest_by_country_at(deaths, '2/20/20'), latest_by_country_at(recovered, '2/20/20')]
combined_12 = pd.concat(informations, axis=1)
combined_12.columns = ["confirmed", "deaths", "recovered"]

sum_up = combined_12.loc["China"]

letality_rate_3 = sum_up["deaths"]/sum_up["confirmed"] * 100

print(letality_rate_3)
display(sum_up)

#**Segunda Parte**

In [ ]:
confirmed.set_index("Country/Region").loc["China"]

In [ ]:
confirmed.set_index("Country/Region").loc["China"].sum().tail()

In [ ]:
confirmed["Country/Region"] == "China"

In [ ]:
is_china = confirmed["Country/Region"] == "China"
is_china.tail()

In [ ]:
confirmed["is_china"]= is_china
confirmed.head()

In [ ]:
confirmed.query("is_china")

In [ ]:
confirmed.groupby("is_china").sum()

In [ ]:
date = '3/20/20'
confirmed.groupby("is_china").sum()[date].plot(kind = 'pie')

In [ ]:
confirmed.groupby("is_china").sum()[date].plot(kind = 'bar')
plt.show()

In [ ]:
import numpy as np

In [ ]:
np.where(confirmed["is_china"], 'China', 'Others')

In [ ]:
confirmed['is_china_label']= np.where(confirmed["is_china"], 'China', 'Others')
display(confirmed.head())
display(confirmed.tail())

In [ ]:
date = '3/2/20'

plt.figure(figsize=(10,5))
confirmed.groupby("is_china_label").sum()[date].plot(kind = 'bar')
plt.title(f"Casos confirmados acumulados até {date}", loc = 'left',pad='25',fontsize = 20 )
plt.show()

In [ ]:
summed = confirmed.groupby("is_china_label").sum()
summed ["delta"] = summed ['3/2/20']- summed ['3/1/20']
summed["delta"].plot(kind='bar')
plt.title(f"Novos casos confirmados em {date}", loc = 'left',pad='25',fontsize = 20 )
plt.show()

In [ ]:
summed.T[2:-2]

In [ ]:
summed.T[2:-2].plot()
plt.title(f"Evolução dos casos com o passar do tempo ", loc = 'left',pad='25',fontsize = 20 )
plt.show()

In [ ]:

differences =summed.T[2:-2].diff().dropna()
differences.plot()
plt.title(f"Casos novos no tempo ", loc = 'left',pad='25',fontsize = 20 )
plt.show()


# Criando novas referências

A partir deste ponto iremos cruzar os dados com outras bases, como a da população mundial, fornecida pela ONU.

In [ ]:
un_population = pd.read_csv('https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv')

In [ ]:
un_population["PopTotal"] = un_population["PopTotal"]*1000
un_population.head()

In [ ]:
un_population.describe()

In [ ]:
un_population.VarID.unique().shape, un_population.VarID.unique(), un_population.Variant.unique()


## Selecionando o VarID mais adequado

A nova base de dados disponibilizada pela ONU inclui pelo menos 14 variantes de população. E para esta análise foram selecionadas as classificadas como **Medium**.

In [ ]:
un_population_2022_med = un_population.query("Time==2022").query("Variant=='Medium'")


In [ ]:
un_population_2022_med


In [ ]:
un_population_2022_med[["Location","PopTotal"]]

In [ ]:
un_population_2022_med = un_population_2022_med[["Location","PopTotal"]].set_index("Location")

In [ ]:
un_population_2022_med

In [ ]:
combined.join(un_population_2022_med)

In [ ]:
combined.join(un_population_2022_med).sort_values("confirmed", ascending=False)[150:198]

In [ ]:
un_population_2022_med.index

In [ ]:
un_population_2022_med.index[0:99], un_population_2022_med.index[100:199], un_population_2022_med.index[200:299], un_population_2022_med.index[300:399], un_population_2022_med.index[400:]

In [ ]:
combined.index[0:49], combined.index[50:99],combined.index[100:149],combined.index[150:198]   

##Engine Python

O query do Pandas nao dispoe do metodo que permita procurar por uma string em determinada coluna e por esta razão podemos utilizar o engine do Python para usar esta funcionalidade.

Como os nomes dos paises na base de dados da ONU não coincidem com aqueles das bases usadas pela OMS, precisaremos investigar como estão organizados a fim de gear os cruzamentos adequados. 

In [ ]:
un_population_2022_med.query("Location.str.contains('China')", engine ='python')

In [ ]:
def rename_location(location):
  if location =="United States of America":
    return 'US'
  if location =="Iran (Islamic Republic of)":
    return "Iran"
  if location =="Russian Federation":
    return "Russia"
  if location =="Bolivia (Plurinational State of)":
    return "Bolivia"    
  if location =="Myanmar":
    return "Burma"
  if location =="State of Palestine":
    return "West Bank and Gaza"
  if location =="Venezuela (Bolivarian Republic of)":
    return "Venezuela"
  if location =="Republic of Moldova":
    return "Moldova"
  if location =="Republic of Korea":
    return "Korea, South"
  if location =="Viet Nam":
    return "Vietnam"
  if location =="Democratic Republic of the Congo":
    return "Congo"
  if location =="Congo":
    return "Congo (Kinshasa)"
  if location =="Côte d'Ivoire":
    return "Cote d'Ivoire"
  if location =="Syrian Arab Republic":
    return "Syria"
  if location =="China, Taiwan Province of China":
    return "Taiwan*"
  if location =="Lao People's Democratic Republic":
    return "Laos"
  if location =="United Republic of Tanzania":
    return "Tanzania"
  if location =="Brunei Darussalam":
    return "Brunei"
  return location

In [ ]:
un_population_2022_med['location_for_who'] = un_population_2022_med.index.map(rename_location)
un_population_2022_med.head()

In [ ]:
un_population_2022_med = un_population_2022_med.set_index("location_for_who")


In [ ]:
un_population_2022_med.head()

In [ ]:
combined_expanded = combined.join(un_population_2022_med).sort_values("confirmed", ascending = False).dropna()
combined_expanded 

In [ ]:
combined_expanded['incidence_ratio'] = combined_expanded['confirmed']/combined_expanded['PopTotal']*100000


In [ ]:
combined_expanded.head(20) 

In [ ]:
combined_expanded['mortality_ratio'] = combined_expanded['deaths']/combined_expanded['PopTotal']*100000

In [ ]:
combined_expanded.head(20) 

In [ ]:
grouped_per_day = pd.DataFrame([confirmed.sum()[3:-1], recovered.sum()[2:], deaths.sum()[2:]], 
             index=["confirmed", "recovered", 'deaths']).T.dropna()

grouped_per_day.head()


In [ ]:
grouped_per_day["letality_rate_1"]= grouped_per_day["deaths"]/grouped_per_day['confirmed']*100
grouped_per_day["letality_rate_2"]= grouped_per_day["deaths"]/(grouped_per_day['recovered']+grouped_per_day["deaths"])*100

In [ ]:
grouped_per_day.head()

In [ ]:
plt.figure(figsize=(12,5)) 
grouped_per_day["letality_rate_1"].plot()
ax = grouped_per_day["letality_rate_2"].plot()
plt.legend(['Mortes/Confirmados', 'Mortes/(Mortes + Recuperados)'])
plt.title("Covid-19 - mortes por casos confirmados no mundo todo", loc = 'left', fontsize = 18, pad = 25)
plt.grid(linestyle = '-')
ax.set_xlabel("Fonte; curso www.alura.com.br e dados Johns Hopkins CSSE https://github.com/CSSEGISandData/COVID-19", fontsize = 10)
ax.set_ylabel("% Letalidade", fontsize = 14)
plt.show()